# Testing Defacing PET images only

There are a few quirks to this code and this notebook is set up to help a user deal with them. For starters, the user first needs to define where each of their datasets are located in the `dataset_dictionary`, this provides common names for later comparison.

```json
dataset_dictionary = {
    "tracer1": "/full/path/to/tracer1dataset/",
    "tracer2": "/full/path/to/tracer2dataset/",
    "tracer3": "/full/path/to/tracer3dataset/"
}
```

Using the datasets above, this notebook will take the user through the defacing of each tracer using 4 different methods:

1. ~~Using the subject's MR image to register w/ and deface it's PET image.~~

2. Using `petdeface/data/sub-01/ses-baseline/anat/sub-01_T1w.nii.gz` as a template to deface each subjects PET images.

3. Using `petdeface/data/sub-mni305/anat/sub-mni305_T1w.nii.gz` ...

4. Creating a "T1" image by averaging the subjects PET image and using that instead to deface.

At the end of this pipeline the user will have ~~16~~12 different datasets.

In [1]:
import logging
from tqdm.notebook import tqdm
import sys
from datetime import datetime
import os
import pathlib

# Set up logging
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, f"petdeface_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler(sys.stdout)
    ]
)

In [7]:
# set up our data

dataset_dictionary = {
    "PS13": "/home/galassiae/Projects/petdeface_no_anat_data/20M0157_MDD_COX1_COX2_BIDS",
    "MC1": "/home/galassiae/Projects/petdeface_no_anat_data/COX-2_MC1_BIDS",
    "TSPO": "/home/galassiae/Projects/petdeface_no_anat_data/TSPO_Brain"
}

home = pathlib.Path.home()
results_dir = home / "petdeface_jupyter_book_results"
results_dir.mkdir(exist_ok=True)


Next we build, install, and import the code we need.

In [3]:
%cd ..
%pip install -e .
# make sure we can import petdeface
from petdeface.petdeface import PetDeface

/home/galassiae/Projects/petdeface


/home/galassiae/.cache/pypoetry/virtualenvs/petdeface-LqsB4PTs-py3.11/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Obtaining file:///home/galassiae/Projects/petdeface
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for petdeface (pyproject.toml) ... done
  Created wheel for petdeface: filename=petdeface-0.2.2-py3-none-any.whl size=5842 sha256=3021cd28e454caceabd10a5b86f4d5d38b0455ea6b0f64fb7a10d71b0ff6cd7a
  Stored in directory: /home/tmp/pip-ephem-wheel-cache-nspf8yxn/wheels/37/dd/a2/64cedb9880fd17ddc66a522d2179134857fd4f8e010f78eb55
Successfully built petdeface
  Attempting uninstall: petdeface
    Found existing installation: petdeface 0.2.2
    Uninstalling petdeface-0.2.2:
      Successfully uninstalled petdeface-0.2.2

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: /home/galassiae/.cache/pypoetry/virtualenvs/petdeface-LqsB4PTs-py3.11/bin/python -m pip install --upgrade pip
Note

In [ ]:
# now we want to run motion correction on each defaced folder that we produced
import subprocess
import os
# We're going to make a mess if we don't use tempfile
from tempfile import TemporaryDirectory
import shutil
import os
#from tqdm.notebook import tqdm
import sys
from datetime import datetime
import logging
import logging
from tqdm.notebook import tqdm
import sys
from datetime import datetime

# next we're going to gather the outputs for a single subject and open them up in freeview
folders_in_results_dir = [os.path.join(results_dir, dir) for dir in os.listdir(results_dir) if 'defaced' in dir]

# collect our freesurfer license
freesurfer_home = os.getenv("FREESURFER_HOME")

# iterate through our datasets with a progress bar
# we don't just append _deface to our presumed outputs because it's assumed that
# we've also placed the original data with T1w's in the output folder as a control
datasets = tqdm(folders_in_results_dir, desc="Motion Corrrecting with petprep_hmc")

for dataset in datasets:
    input = dataset
    output = os.path.join(dataset, 'derivatives', 'petprep_hmc')
    freesurfer_license = os.path.join(freesurfer_home, 'license.txt')
    # only run if there aren't outputs for petprep_hmc in the output folder
    if len(os.listdir(output)) != 0:
        command = f"docker run --rm -v {input}:/data/input -v {output}:/data/output -v {freesurfer_license}:/opt/freesurfer/license.txt martinnoergaard/petprep_hmc:latest /data/input /data/output/ participant --n_procs 16"
        print(f"Running petprep_hmc on {output} with command {command}")
        subprocess.run(command, shell=True)
    else:
        print(f"{output} contains some results, delete this directory to rerun petprep_hmc")

Motion Corrrecting with petprep_hmc:   0%|          | 0/10 [00:00<?, ?it/s]

/home/galassiae/petdeface_jupyter_book_results/PS13_t1_defaced/derivatives/petprep_hmc contains some results, delete this directory to rerun petprep_hmc
Running petprep_hmc on /home/galassiae/petdeface_jupyter_book_results/PS13_pet_defaced/derivatives/petprep_hmc with command docker run --rm -v /home/galassiae/petdeface_jupyter_book_results/PS13_pet_defaced:/data/input -v /home/galassiae/petdeface_jupyter_book_results/PS13_pet_defaced/derivatives/petprep_hmc:/data/output -v /usr/local/freesurfer/7.4.1/license.txt:/opt/freesurfer/license.txt martinnoergaard/petprep_hmc:latest /data/input /data/output/ participant --n_procs 16
FSL is installed at: /opt/fsl
250508-03:32:52,559 nipype.workflow INFO:
	 Workflow petprep_hmc_wf settings: ['check', 'execution', 'logging', 'monitoring']
250508-03:32:52,582 nipype.workflow INFO:
	 Running in parallel.
250508-03:32:52,585 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 3 jobs ready. Free memory (GB): 338.43/338.43, Free processors: 16/16

In [116]:
# collect all of the nifti's in the output directory for grouping and comparison
import nibabel
import os
import re

all_motion_files = []
print(f"Checking {results_dir} folder for motion files...")
for root, folders, files in os.walk(results_dir):
    # collect all the nifti's in the folder
    for f in files:
        if 'confound' in f and not '_wf' in f:
            all_motion_files.append(os.path.join(root, f))

        
# now we group all the nifti's by subject
subject_files = {}
for n in all_motion_files:
    match = re.search(r'sub-[a-zA-Z0-9]+(?=_)', str(n))
    if match:
        sub_id = match.group(0)
        if sub_id not in subject_files:
            subject_files[sub_id] = []
        subject_files[sub_id].append(n)

from pprint import pprint


# next we group by tracer and sessions
subjects_grouped_by_tracer = {}
for subject, cofounds in subject_files.items():
    subjects_grouped_by_tracer[subject] = {}
    for n in cofounds:
        tracer_match = re.search(r'trc-[a-zA-Z0-9]+(?=_)', str(n))
        if tracer_match:
            tracer_match = tracer_match.group(0)
            if not subjects_grouped_by_tracer[subject].get(tracer_match):
                subjects_grouped_by_tracer[subject][tracer_match] = {}
            session_match = re.search(r'ses-[a-zA-Z0-9]+(?=_)', str(n))
            if session_match:
                session_match = session_match.group(0)
                if not subjects_grouped_by_tracer[subject][tracer_match].get(session_match):
                    subjects_grouped_by_tracer[subject][tracer_match][session_match] = []
                subjects_grouped_by_tracer[subject][tracer_match][session_match].append(n)
            
            
# great we've collected all of our different templated runs and organized them by subject and tracer.            
import numpy as np
import pandas as pd
from copy import deepcopy
templates = ['original', 't1', 'mni', 'pet']
for subject, tracers in subjects_grouped_by_tracer.items():
    print(subject)
    for tracer in tracers:
        print(tracer)
        storage = {}
        for session, tsvs in subjects_grouped_by_tracer[subject][tracer].items():
            print(session)
            tsvs = [pathlib.Path(tsv) for tsv in tsvs]
            for tsv in tsvs:
                storage[tsv.parts[4]] = pd.read_csv(tsv,delimiter='\t', index_col=0)
        for key, value in storage.items():
            print(key)
            print(value)
        # create a correlation matrix for each set
        #df = pd.DataFrame(storage, columns=storage.keys())
        #print(df)
        #cor_mat = df.corr()
        #print(cor_mat)
        #import matplotlib.pyplot as plt
        #fig = plt.matshow(cor_mat)
        #plt.savefig(f"{subject}_{tracer}_plot.png")
        


Checking /home/galassiae/petdeface_jupyter_book_results folder for motion files...
sub-01
trc-ps13
ses-baseline
PS13_t1_defaced
     trans_x   trans_y   trans_z     rot_x     rot_y     rot_z     max_x  \
0  -1.363508  2.371787  0.551511 -0.008080 -0.004122  0.006460  1.023825   
1  -1.363508  2.371787  0.551511 -0.008080 -0.004122  0.006460  1.023825   
2  -1.363508  2.371787  0.551511 -0.008080 -0.004122  0.006460  1.023825   
3  -1.363508  2.371787  0.551511 -0.008080 -0.004122  0.006460  1.023825   
4  -1.363508  2.371787  0.551511 -0.008080 -0.004122  0.006460  1.023825   
5  -0.839242  1.819760  0.308121 -0.006928 -0.001142  0.005219  0.569450   
6  -0.675669  0.659301  0.769098 -0.004038 -0.001630  0.001557  0.584148   
7  -0.780503  0.520542  1.110373 -0.003032 -0.002374  0.001066  0.721122   
8  -0.312349 -0.050955  1.105456 -0.000919 -0.000668 -0.000769  0.417040   
9  -0.414496  0.065474  1.154053 -0.000006  0.000424  0.000162  0.442102   
10 -0.254852  0.044870  1.108963  0.

Default to using 12 dof for new methods and add to cli

Update dof to be 12 forthis

# Show Images
## Tracers
## Different Subjects

# Show Motion Correction

